# AI stock analysis

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PauliusU/AI-stock-analysis/blob/master/AI-stock-analysis.ipynb)

❗Work in progress, some cells may not work❗

Goal for MWE is to train first stock price prediction model

## Get data

In [ ]:
# Use Yahoo Finance's API wrapper
!pip install yfinance
import yfinance as yf

print(f"yfinance version: {yf.__version__}")

In [ ]:
# Get stock data for the last 5 years
ticker = 'TSLA'
stock_data = yf.download(ticker, start='2017-11-01', end='2022-11-01')
print(stock_data.head())
print(stock_data.tail())

## Plot data

In [ ]:
# Plot closing price over time

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.title('Historical stock prices')
plt.plot(stock_data['Close'])
plt.xlabel('Time')
plt.ylabel('Closing price in USD')

## Prepare training and test sets

To prepare data to train and test LTSM model, only the closing price would be needed. Cell below gets 80% of these data points for closing price, normalizes them, and converts them into 2D array.

In [ ]:
# Prepare training set

import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

close_prices = stock_data['Close']
values = close_prices.values
training_data_len = math.ceil(len(values) * 0.8)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(values.reshape(-1, 1))
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i - 60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [ ]:
# Prepare test set

test_data = scaled_data[training_data_len - 60:, :]
x_test = []
y_test = values[training_data_len:]

for i in range(60, len(test_data)):
    x_test.append(test_data[i - 60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

## Set up model

In [ ]:
# Check if GPU is used
import tensorflow as tf

print(
    f"Number of GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

# Print the list of available training devices - alternative method to verify TensorFlow sees the GPU
print(tf.config.list_physical_devices())

In [ ]:
# Set up LSTM network architecture

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(
    layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(layers.LSTM(100, return_sequences=False))
model.add(layers.Dense(25))
model.add(layers.Dense(1))
model.summary()

# Set optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model

model.fit(x_train, y_train, batch_size=1, epochs=3)

In [ ]:
# Evaluation

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test) ** 2)
print(rmse)

In [ ]:
# Visualize the predicted prices

data = stock_data.filter(['Close'])
train = data[:training_data_len]
validation = data[training_data_len:]
validation['Predictions'] = predictions
plt.figure(figsize=(14, 7))
plt.title('Predicted prices')
plt.xlabel('Date')
plt.ylabel('Closing price in USD')
plt.plot(train)
plt.plot(validation[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()